In [100]:
import numpy as np
import copy

from gym_tictactoe.env import TicTacToeEnv


env = TicTacToeEnv(size=3)

def evaluate(board):
    """
    Evaluate the current state of the board.
    Returns:
    - 1 if the maximizing player wins
    - -1 if the minimizing player wins
    - 0 if it's a tie or the game is ongoing
    """
    n = int(np.sqrt(len(board)))

    # Check rows and columns
    for i in range(n):
        if all(board[i * n + j] == 1 for j in range(n)):
            return 1
        if all(board[j * n + i] == 1 for j in range(n)):
            return 1
        if all(board[i * n + j] == 2 for j in range(n)):
            return -1
        if all(board[j * n + i] == 2 for j in range(n)):
            return -1

    # Check diagonals
    if all(board[i * (n + 1)] == 1 for i in range(n)):
        return 1
    if all(board[i * (n - 1) + (n - 1)] == 1 for i in range(n)):
        return 1
    if all(board[i * (n + 1)] == 2 for i in range(n)):
        return -1
    if all(board[i * (n - 1) + (n - 1)] == 2 for i in range(n)):
        return -1

    # Check for a tie
    if all(cell != 0 for cell in board):
        return 0

    # Game still ongoing
    return None

def abminimax(board, depth, alpha, beta, maximizingPlayer):
    """
    Implementation of the alpha-beta pruning minimax algorithm.
    """
    score = evaluate(board)
    # print(score)

    if score is not None:
        return score

    if maximizingPlayer:
        maxEval = -np.inf
        for i in range(len(board)):
            if board[i] == 0:
                new_board = copy.deepcopy(board)
                new_board[i] = 1
                eval = abminimax(new_board, depth + 1, alpha, beta, False)
                maxEval = max(maxEval, eval)
                alpha = max(alpha, eval)
                if beta <= alpha:
                    break
        return maxEval
    else:
        minEval = np.inf
        for i in range(len(board)):
            if board[i] == 0:
                new_board = copy.deepcopy(board)
                new_board[i] = 2
                eval = abminimax(new_board, depth + 1, alpha, beta, True)
                minEval = min(minEval, eval)
                beta = min(beta, eval)
                if beta <= alpha:
                    break
        return minEval

def find_best_move(board):
    """
    Finds the best move using the alpha-beta pruning minimax algorithm.
    """
    print(board)
    best_val = -np.inf
    best_move = None
    alpha = -np.inf
    beta = np.inf

    for i in range(len(board)):
        if board[i] == 0:
            new_board = copy.deepcopy(board)
            new_board[i] = 1
            move_val = abminimax(new_board, 0, alpha, beta, False)

            if move_val > best_val:
                best_move = i
                best_val = move_val
            alpha = max(alpha, move_val)

    return best_move

In [101]:
# Example of how to use the minimax algorithm with the TicTacToe environment
from stable_baselines3 import DQN, DDPG, PPO

env.reset()
done = False
model = PPO.load("ppomodel/best_model.zip")
observation = np.asarray([0] * 9)
while not done:
    # Human player's turn
    print("PPO's turn!")
    action, _states = model.predict(observation, deterministic=True)
    observation, reward, done, trunc, _ = env.step(action)
    env.render()

    if done:
        print("Game Over!")
        break
    
    # Agent's turn using minimax
    print("Alpha-Beta's turn!")
    best_move = np.asarray(find_best_move(observation))
    observation, reward, done, _, _ = env.step(best_move)
    env.render()
    
    if done:
        print("Game Over!")
        break

    

PPO's turn!
   | | 
  ------
   | | 
  ------
   |O| 
Alpha-Beta's turn!
[0. 0. 0. 0. 0. 0. 0. 2. 0.]
   |X| 
  ------
   | | 
  ------
   |O| 
PPO's turn!
   |X| 
  ------
   | | 
  ------
   |O|O
Alpha-Beta's turn!
[0. 1. 0. 0. 0. 0. 0. 2. 2.]
   |X| 
  ------
   | | 
  ------
  X|O|O
PPO's turn!
   |X| 
  ------
   | |O
  ------
  X|O|O
Alpha-Beta's turn!
[0. 1. 0. 0. 0. 2. 1. 2. 2.]
   |X|X
  ------
   | |O
  ------
  X|O|O
PPO's turn!
   |X|X
  ------
   |O|O
  ------
  X|O|O
Alpha-Beta's turn!
[0. 1. 1. 0. 2. 2. 1. 2. 2.]
  X|X|X
  ------
   |O|O
  ------
  X|O|O
Game Over!


In [102]:
# # Example of how to use the minimax algorithm with the TicTacToe environment
# env.reset()
# done = False
# while not done:
#     # Human player's turn
#     # env.render()
#     print("Your turn!")
#     n = env.size
#     action = np.asarray(int(input("Enter action: ")))
#     observation, reward, done, _, _ = env.step(action)
#     env.render()

#     if done:
#         print("Game Over!")
#         break

#     # Agent's turn using minimax
#     print("Agent's turn!")
#     print(observation)
#     best_move = np.asarray(find_best_move(observation))
#     observation, reward, done, _, _ = env.step(best_move)
    
#     env.render()
    
#     if done:
#         print("Game Over!")
#         break